##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Examinando o grafo do TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/graphs"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/graphs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/graphs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tensorboard/graphs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

O **painel de controle Graphs** (Grafos) do TensorBoard é uma ferramenta avançada para examinar seu modelo do TensorFlow. Você pode ver rapidamente um grafo conceitual da estrutura do seu modelo e garantir que ele corresponda ao design pretendido. Também é possível visualizar um grafo no nível da op para entender como o TensorFlow compreende seu programa. Ao examinar o grafo no nível da op, você ganha insights sobre como mudar seu modelo. Por exemplo, é possível reformular seu modelo se o treinamento estiver progredindo mais lentamente do que o esperado.

Este tutorial apresenta uma rápida visão geral de como gerar os dados de diagnóstico dos grafos e visualizá-los no painel de controle Graphs do TensorBoard. Você definirá e treinará um modelo Sequential simples do Keras para o dataset Fashion-MNIST e aprenderá a registrar e examinar os grafos do seu modelo. Você também usará uma API de tracing para gerar dados de grafos para funções criadas usando a nova anotação `tf.function`.

## Configuração

In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.2.0


In [4]:
import tensorboard
tensorboard.__version__

'2.2.1'

In [5]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Defina um modelo Keras

Neste exemplo, o classificador é um modelo Sequential de quatro camadas.

In [6]:
# Define the model.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

Baixe e prepare os dados de treinamento.

In [7]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0

## Treine o modelo e registre os dados

Antes do treinamento, defina a [callback do TensorBoard para o Keras](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard), especificando o diretório do log. Ao passar essa callback para Model.fit(), você garante que os dados do grafo sejam registrados para visualização no TensorBoard.

In [8]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

Epoch 1/5
938/938 [==============================] - 2s 2ms/step - loss: 0.6955 - accuracy: 0.7618
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4877 - accuracy: 0.8296
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4458 - accuracy: 0.8414
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4246 - accuracy: 0.8476
Epoch 5/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4117 - accuracy: 0.8508


## Grafo no nível da op

Inicie o TensorBoard e aguarde alguns segundos para a interface do usuário carregar. Selecione o painel de controle Graphs ao tocar em "Graphs" na parte superior. 

In [ ]:
%tensorboard --logdir logs

Opcionalmente, você também pode usar o TensorBoard.dev para criar um experimento hospedado e compartilhável. 

In [ ]:
!tensorboard dev upload \
  --logdir logs \
  --name "Sample op-level graph" \
  --one_shot

Por padrão, o TensorBoard mostra o **grafo no nível da op**. (À esquerda, você pode ver a tag "Default" selecionada.) Observe que o grafo está invertido: os dados fluem de baixo para cima, então está de cabeça para baixo em comparação com o código. No entanto, é possível ver que o grafo corresponde bastante à definição de modelo do Keras, com bordas adicionais para outros nós de computação.

Geralmente, os grafos são muito grandes, então você pode manipular a visualização deles:

- Role para aumentar e diminuir o **zoom**
- Arraste para **girar panoramicamente**
- Clique duas vezes para ativar a **expansão de nó** (um nó pode ser um container para outros nós)

Você também pode ver os metadados ao clicar em um nó. Assim, é possível conferir entradas, saídas, formatos e outros detalhes.


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation.png?raw=1"/> -->

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation_detail.png?raw=1"/> -->

## Grafo conceitual

Além do grafo de execução, o TensorBoard também mostra um **grafo conceitual**. É uma visualização com apenas o modelo do Keras. Isso pode ser útil se você estiver reutilizando um modelo salvo e quer examinar ou validar sua estrutura.

Para ver o grafo conceitual, selecione a tag "keras". Para esse exemplo, você verá um nó **Sequential** recolhido. Clique duas vezes no nó para ver a estrutura do modelo:

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_tag_selection.png?raw=1"/> --> <br/>
<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_conceptual.png?raw=1"/> -->

## Grafos de tf.functions

Até agora, os exemplos descreveram grafos de modelos Keras, onde os grafos foram criados ao definir camadas do Keras e chamar Model.fit().

Talvez você se depare com uma situação em que precise usar a anotação `tf.function` para ["autografar"](https://www.tensorflow.org/guide/function), ou seja, transformar uma função de computação Python em um grafo do TensorFlow de alto desempenho. Para essas situações, use a **API Summary Trace do TensorFlow** a fim de registrar funções autografadas para visualização no TensorBoard.

Para usar a API Summary Trace:

- Defina e anote uma função com `tf.function`.
- Use `tf.summary.trace_on()` imediatamente antes do local de chamada da função.
- Adicione as informações do perfil (memória, tempo de CPU) ao grafo passando `profiler=True`.
- Com um escritor de arquivo de resumo, chame `tf.summary.trace_export()` para salvar os dados de log.

Em seguida, você pode usar o TensorBoard para ver o comportamento da sua função.


In [10]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

In [ ]:
%tensorboard --logdir logs/func

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_autograph.png?raw=1"/> -->

Agora você pode ver a estrutura da sua função conforme compreendida pelo TensorBoard. Clique no botão de opção "Profile" para ver estatísticas de CPU e memória.